# Neural Networks - Backpropagation and Computational Graph

## Simple Graph with One Parameter
Let's start with simple computation graph. Consider composite function $f(x)$ as follows:

$$f(x) = ((x + 5)^3+1)^2$$

And let's compute our function $f(x)$ at the point $x = -3$. So, if we decompose our function $f(x)$ into simple functions, we get:

$$h= x + 5 = 2$$
$$g = h^3 + 1 = 9$$
$$f = g^2 = 81$$

See [Computational Graph]() for more details.

We'll use PyTorch to compute the output (forward propagation) of our function $f(x)$ at the point $x = -3$. Then we will compute the gradients (backward propagation) of our function $f(x)$ with respect to $x$.

In [1]:
import torch

In [2]:
x = torch.tensor(-3.0, requires_grad=True)

Since we want to compute the gradient (derivative) of our function $f(x)$ with respect to $x$, we need to set `requires_grad=True` for any parameter that will be part of the computation graph for which we want to compute the gradient.

In [3]:
f_x = ((x + 5) ** 3 + 1) ** 2
print(f_x.data)

tensor(81.)


In [4]:
f_x.backward()

# Gradients
print(f"df/dx: {x.grad}")

df/dx: 216.0


During the forward propagation, PyTorch automatically builds a directed acyclic graph (DAG) of operations. This is when we calculate `f_x`. PyTorch create this graph when `torch.tensor` with `requires_grad=True` is involved in the computation.

When we call `f_x.backward()`, it walks backward through the computational graph to to each parameter (leaf node) and apply the chain rule of calculus to compute the gradients of the output (the final node where we called `backward()`) with respect to each parameter.

Gradient Accumulation: The calculated gradients are stored in the `grad` attribute of each parameter tensor that has `requires_grad=True`.

This [PyTorch Computation Graph](https://www.youtube.com/watch?v=MswxJw-8PvE) is a good video on this topic.

## Graph with Multiple Parameters

Now let's consider a simple linear regression model with one input feature $x$ and one target $y$. The model is defined as follows:

$$f_{w,b}(x) = wx + b$$

Where $w$ is the weight and $b$ is the bias. 

The [cost function]() is defined as the mean squared error (MSE) for a dataset with only one sample:

$$J(w,b) = \frac{1}{2}(f_{w,b}(x) - y)^2$$


[Gradient Descent]() is the most common optimization algorithm used to minimize the cost function. The algorithm works by iteratively updating the parameters in the opposite direction of the gradients of the cost function with respect to the parameters. So, we need to compute the gradients of the cost function with respect to the parameters $w$ and $b$.
 

See [Computational Graph]() for more details.


Let's define our input $x$, target value $y$, weight $w$ and bias $b$ as follows:

In [5]:
x = torch.tensor(-3.0)
y = torch.tensor(5.0)
w = torch.tensor(3.0, requires_grad=True)
b = torch.tensor(1.0, requires_grad=True)

We didn't set the `requires_grad=True` for $x$ and $y$ because they are the input and target values. They are not parameters of the model which we need to optimize by computing the gradients.

### Step 1 - Forward Propagation (Compute Model Output)
We'll compute the model's output

In [6]:
# Forward Propagation (compute the output)
c = w * x
f = c + b

### Step 2 - Forward Propagation (Compute the Cost Function)

In [7]:
# Compute the Cost
d = f - y
J = (d**2) / 2

In PyTorch, the intermediate gradients for non-leaf nodes are not stored by default which is more efficient in practice when we have a large number of parameters. However, in this example to demonstrate the backward steps in the computational graph, we will change this behavior by calling `retain_grad()` on the intermediate tensors.

In [8]:
c.retain_grad()
f.retain_grad()
d.retain_grad()

In [9]:
print(f"Input feature x and target y:\nx: {x.data}\ny: {y.data}\n")
print(f"Model parameters:\nw: {w.data}\nb: {b.data}\n")
print(f"Model output:\nc: {c.data}\nf: {f.data}\n")
print(f"Cost:\nd: {d.data}\nJ: {J.data}\n")

Input feature x and target y:
x: -3.0
y: 5.0

Model parameters:
w: 3.0
b: 1.0

Model output:
c: -9.0
f: -8.0

Cost:
d: -13.0
J: 84.5



### Step 3 - Backpropagation (Compute the Gradients)

In [10]:
# Backpropagation (compute the gradients)
J.backward()

In [11]:
print(f"dJ/dd: {d.grad}")
print(f"dJ/df: {f.grad}")
print(f"dJ/dc: {c.grad}")
print(f"dJ/db: {b.grad}")
print(f"dJ/dw: {w.grad}")

dJ/dd: -13.0
dJ/df: -13.0
dJ/dc: -13.0
dJ/db: -13.0
dJ/dw: 39.0


## Simple Neural Network
Let's now see the computational graph for a simple neural network with the following architecture:
- Layer 0 (input layer): 2 features
- Layer 1: Fully connected layer with 3 neurons and ReLU activation function.
- Layer 2: Fully connected layer with 2 neurons and ReLU activation function.
- Layer 3 (output): Fully connected layer with 1 neuron (output) and Sigmoid activation function.

In this example we use a batch dataset with 2 samples. The input $X$ and target $Y$ are defined as follows:

$$X = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}$$
$$Y = \begin{bmatrix} 0 \\ 1 \end{bmatrix}$$

Which means, for example 1 $x_1 = 1$ and $x_2 = 2$ and the target class $y = 0$.

Recall that we maintain each sample in **rows** and features in **columns**. So, each row of $X$ and $Y$ is associated with one sampleDataset with .

In [12]:
X = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = torch.tensor([[0.0], [1.0]])

Let's create our neural network

In [13]:
import torch.nn as nn
import torch.nn.functional as F


class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()

        # Define the model architecture (Layers and nodes)
        self.linear1 = nn.Linear(in_features=2, out_features=3)
        self.linear2 = nn.Linear(in_features=3, out_features=2)
        self.linear3 = nn.Linear(in_features=2, out_features=1)

    def forward(self, x):
        # Forward Propagation happens here.
        # It takes the input tensor x and returns the output tensor for each
        # layer by applying the linear transformation first and then the
        # activation function.
        # It start from layer 1 and goes forward layer by layer to the output
        # layer.

        # Layer 1 linear transformation
        Z1 = self.linear1(x)
        # Layer 1 activation
        A1 = F.relu(Z1)

        # Layer 2 linear transformation
        Z2 = self.linear2(A1)
        # Layer 2 activation
        A2 = F.relu(Z2)

        # Layer 3 (output layer) linear transformation
        Z3 = self.linear3(A2)
        # Layer 3 activation
        A3 = F.sigmoid(Z3)

        # Print the intermediate results
        print(
            f"Z1:\n{Z1}\nA1:\n{A1}\nZ2:\n{Z2}\nA2:\n{A2}\nZ3:\n{Z3}\nA3:\n{A3}"
        )

        # Output of the model (prediction)
        return A3

In practice, for binary classification problems, when we use the Sigmoid activation function in the output layer, we defer the computation of output layer activation to outside the model. In that way, the model only outputs the **logits** of the output layer. Then the logits are passed to the Sigmoid activation function to get the predicted probabilities, or for training to the  `BCEWithLogitsLoss` binary cross-entropy loss function which combines the Sigmoid activation function and the binary cross-entropy loss.

That approach is more numerically stable and typically gives better training results. However, in this example, for easier demonstration, we will include the Sigmoid activation function in the model's output layer and use the `BCELoss` loss function. 

In [14]:
model = NeuralNet()
print(model)

NeuralNet(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (linear2): Linear(in_features=3, out_features=2, bias=True)
  (linear3): Linear(in_features=2, out_features=1, bias=True)
)


Let's see the initial weights and biases of our neural network.

In [15]:
def print_model_parameters(model):
    for i, child in enumerate(model.children()):
        print(f"Layer {i+1}: {type(child).__name__}")
        child_parameters = dict(child.named_parameters())

        for name, param in child_parameters.items():
            print(f"\n{name}: {param.size()} {param}")
            print(f"{name}.grad:\n{param.grad}")

        print("-" * 80)


print_model_parameters(model)

Layer 1: Linear

weight: torch.Size([3, 2]) Parameter containing:
tensor([[-0.0669,  0.6420],
        [-0.5951,  0.0679],
        [-0.7008,  0.4456]], requires_grad=True)
weight.grad:
None

bias: torch.Size([3]) Parameter containing:
tensor([-0.3008, -0.3751,  0.1771], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 2: Linear

weight: torch.Size([2, 3]) Parameter containing:
tensor([[ 0.5553,  0.5280, -0.5235],
        [-0.4386, -0.1877, -0.3298]], requires_grad=True)
weight.grad:
None

bias: torch.Size([2]) Parameter containing:
tensor([-0.4900,  0.4226], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 3: Linear

weight: torch.Size([1, 2]) Parameter containing:
tensor([[ 0.1257, -0.4513]], requires_grad=True)
weight.grad:
None

bias: torch.Size([1]) Parameter containing:
tensor([-0.5929], requires_grad=True)
bias.grad:
None
-------------

As we expect, gradients of parameters are `None` since we haven't computed any gradients yet.

For this example for simplicity and having reproducible results, we'll set the weights and biases manually. Let's say we have the following weights and biases:

Similar to the way that PyTorch creates weights matrices:
- Each row of $W^{[l]}$ is associated with one neuron in the layer $l$. For example, in layer 1, We have 3 neurons, so we have 3 rows in $W^{[1]}$.
- Each column is associated with one feature of input values. For example, the number of columns in $W^{[1]}$ is equal to the number of features in the input layer. We have 2 features in the input layer $X$, so we have 2 columns in $W^{[1]}$, and so on.

Note: The number of weight and biases are independent of the number of samples in the batch or entire dataset. The whole point is to optimize these parameters by exposing them to the entire dataset through cycle of forward and backward propagation. So, no matter what is the size of the dataset, the number of parameters in the model is fixed and defined by the architecture of the neural network.

**Layer 1 (3 neurons):**
$$W^{[1]} = \begin{bmatrix} -1 & 2 \\ 3 & 0.5 \\ -0.1 & -4\end{bmatrix} \quad {\vec{\mathbf{b}}}^{[1]} = \begin{bmatrix} 1 & -2 & 0.3 \end{bmatrix}$$


**Layer 2 (2 neurons):**
$$W^{[2]} = \begin{bmatrix} 0.5 & 1 & -2 \\ 0.7 & 0.1 & 0.3\end{bmatrix} \quad {\vec{\mathbf{b}}}^{[2]} = \begin{bmatrix} -4 & 5 \end{bmatrix}$$

**Layer 3 (output):**
$$W^{[3]} = \begin{bmatrix} 0.5 & -0.3 \end{bmatrix} \quad {\vec{\mathbf{b}}}^{[3]} = \begin{bmatrix} 0.1 \end{bmatrix}$$ 


In [16]:
# Layer 1
W_1 = torch.tensor([[-1.0, 2.0], [3.0, 0.5], [-0.1, -4.0]], requires_grad=True)
b_1 = torch.tensor([1.0, -2.0, 0.3], requires_grad=True)

# Layer 2
W_2 = torch.tensor([[0.5, 1.0, -2.0], [0.7, 0.1, 0.3]], requires_grad=True)
b_2 = torch.tensor([-4.0, 5.0], requires_grad=True)

# Layer 3 (Output layer)
W_3 = torch.tensor([[0.5, -0.3]], requires_grad=True)
b_3 = torch.tensor([0.1], requires_grad=True)

In [17]:
model.linear1.weight.data.copy_(W_1)
model.linear1.bias.data.copy_(b_1)

model.linear2.weight.data.copy_(W_2)
model.linear2.bias.data.copy_(b_2)

model.linear3.weight.data.copy_(W_3)
model.linear3.bias.data.copy_(b_3)

print_model_parameters(model)

Layer 1: Linear

weight: torch.Size([3, 2]) Parameter containing:
tensor([[-1.0000,  2.0000],
        [ 3.0000,  0.5000],
        [-0.1000, -4.0000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([3]) Parameter containing:
tensor([ 1.0000, -2.0000,  0.3000], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 2: Linear

weight: torch.Size([2, 3]) Parameter containing:
tensor([[ 0.5000,  1.0000, -2.0000],
        [ 0.7000,  0.1000,  0.3000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([2]) Parameter containing:
tensor([-4.,  5.], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 3: Linear

weight: torch.Size([1, 2]) Parameter containing:
tensor([[ 0.5000, -0.3000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([1]) Parameter containing:
tensor([0.1000], requires_grad=True)
bias.grad:
None
----------------------

Now let's run the forward propagation using the current weights and biases, and the input $X$.

In [18]:
# Forward Propagation
output = model(X)

Z1:
tensor([[  4.0000,   2.0000,  -7.8000],
        [  6.0000,   9.0000, -16.0000]], grad_fn=<AddmmBackward0>)
A1:
tensor([[4., 2., 0.],
        [6., 9., 0.]], grad_fn=<ReluBackward0>)
Z2:
tensor([[ 0.0000,  8.0000],
        [ 8.0000, 10.1000]], grad_fn=<AddmmBackward0>)
A2:
tensor([[ 0.0000,  8.0000],
        [ 8.0000, 10.1000]], grad_fn=<ReluBackward0>)
Z3:
tensor([[-2.3000],
        [ 1.0700]], grad_fn=<AddmmBackward0>)
A3:
tensor([[0.0911],
        [0.7446]], grad_fn=<SigmoidBackward0>)


In [ ]:
cost = F.binary_cross_entropy(output, Y)

print(f"Cost: {cost}")

Loss: 0.19522885978221893


In [20]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.01)

The `zero_grad()` simply resets (zeros out) the gradients of all parameters. Here, we don't need it since we haven't computed any gradients yet, and the gradients are `None`, however this is a good practice to reset the gradients before any new gradients computation.

In [21]:
optimizer.zero_grad()

In [22]:
print_model_parameters(model)

Layer 1: Linear

weight: torch.Size([3, 2]) Parameter containing:
tensor([[-1.0000,  2.0000],
        [ 3.0000,  0.5000],
        [-0.1000, -4.0000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([3]) Parameter containing:
tensor([ 1.0000, -2.0000,  0.3000], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 2: Linear

weight: torch.Size([2, 3]) Parameter containing:
tensor([[ 0.5000,  1.0000, -2.0000],
        [ 0.7000,  0.1000,  0.3000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([2]) Parameter containing:
tensor([-4.,  5.], requires_grad=True)
bias.grad:
None
--------------------------------------------------------------------------------
Layer 3: Linear

weight: torch.Size([1, 2]) Parameter containing:
tensor([[ 0.5000, -0.3000]], requires_grad=True)
weight.grad:
None

bias: torch.Size([1]) Parameter containing:
tensor([0.1000], requires_grad=True)
bias.grad:
None
----------------------

In [23]:
# Backpropagation (compute the gradients)
cost.backward()

In [24]:
print_model_parameters(model)

Layer 1: Linear

weight: torch.Size([3, 2]) Parameter containing:
tensor([[-1.0000,  2.0000],
        [ 3.0000,  0.5000],
        [-0.1000, -4.0000]], requires_grad=True)
weight.grad:
tensor([[-0.0249, -0.0396],
        [-0.1814, -0.2428],
        [ 0.0000,  0.0000]])

bias: torch.Size([3]) Parameter containing:
tensor([ 1.0000, -2.0000,  0.3000], requires_grad=True)
bias.grad:
tensor([-0.0147, -0.0614,  0.0000])
--------------------------------------------------------------------------------
Layer 2: Linear

weight: torch.Size([2, 3]) Parameter containing:
tensor([[ 0.5000,  1.0000, -2.0000],
        [ 0.7000,  0.1000,  0.3000]], requires_grad=True)
weight.grad:
tensor([[-0.3831, -0.5747,  0.0000],
        [ 0.1752,  0.3175,  0.0000]])

bias: torch.Size([2]) Parameter containing:
tensor([-4.,  5.], requires_grad=True)
bias.grad:
tensor([-0.0639,  0.0246])
--------------------------------------------------------------------------------
Layer 3: Linear

weight: torch.Size([1, 2]) Parame